In [ ]:
!apt install mongodb >log
!service mongodb start



 * Starting database mongodb
   ...done.


In [ ]:
!wget https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz
!gunzip amazon-meta.txt.gz

--2021-06-30 19:20:23--  https://snap.stanford.edu/data/bigdata/amazon/amazon-meta.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210807517 (201M) [application/x-gzip]
Saving to: ‘amazon-meta.txt.gz’

amazon-meta.txt.gz  100%[===================>] 201.04M  8.97MB/s    in 14s     

2021-06-30 19:20:37 (14.5 MB/s) - ‘amazon-meta.txt.gz’ saved [210807517/210807517]



In [ ]:
def limpeza(text): #Criando estrutura de dados para o arquivo da amazon
  text = text.split('\n')
  # print(text)
  flag = 0
  vet = {}
  for words in text:
    # print(words)
    #words = words.rstrip()

    if words.startswith('Id:'):
      vet['id'] = words.split()[1].strip()

    elif words.startswith('ASIN:'):
      vet['asin'] = words.split('ASIN:')[1].strip()
        
    elif words.startswith('  title:'):
      vet['title'] = words.split('title: ')[1].strip()

    elif words.startswith('  group:'):
      vet['group'] = words.split('group:')[1].strip()

    elif words.startswith('  salesrank:'):
      vet['salesrank'] = words.split('salesrank:')[1].strip()

    elif words.startswith('  similar:'):
      if words.strip()[9] != 0:
        vet['similar'] = words.split()[2:]
      
    elif words.startswith('  categories:'):
      vet['categories'] = []

    elif words.startswith('   |'):
      categories = words.split('|')
      if len(categories) == 1:
          pass
      else:
          vet['categories'].append(categories[len(categories)-1])

    elif words.startswith('  reviews:'):
      vet['avg_rating'] =  words.split(': ')[4]
      vet['reviews'] = []

    elif words.startswith('    2') or words.startswith('    1'):
      aux = words.split()
      vet['reviews'].append({'data':aux[0], 'cliente':aux[2], 
                             'rating':aux[4], 'votes':aux[6], 'helpful':aux[8]})
  #print(vet)      
  return vet

def delimited(file, delimiter, bufsize=4096):
  buf = ''
  while True:
    newbuf = file.read(bufsize)
    if not newbuf:
      yield buf
      return
    buf += newbuf
    lines = buf.split(delimiter)
    for line in lines[:-1]:
      yield line
    buf = lines[-1]


In [ ]:
from pymongo import MongoClient
client = MongoClient()

client.list_database_names()

['admin', 'local']

In [ ]:
db = client['database']
products = db['products']

In [ ]:
data = list(delimited(open('amazon-meta.txt', 'r'), '\n\n'))

In [ ]:
#Limpeza dos dados 
cont = 0
vetor = []
for produto in data:
  if (not produto.startswith('#') and not produto == 'Total'):
    x = limpeza(produto)
    vetor.append(x)


In [ ]:
#Inserindo dentro do banco de dados
cont = 0
for i in vetor:
  cont+=1
  if cont > 1:
    products.insert_one(i)
vetor[1]

{'_id': ObjectId('60dcc4398983eee3578b1823'),
 'asin': '0827229534',
 'avg_rating': '5',
 'categories': ['Preaching[12368]', 'Sermons[12370]'],
 'group': 'Book',
 'id': '1',
 'reviews': [{'cliente': 'A2JW67OY8U6HHK',
   'data': '2000-7-28',
   'helpful': '9',
   'rating': '5',
   'votes': '10'},
  {'cliente': 'A2VE83MZF98ITY',
   'data': '2003-12-14',
   'helpful': '5',
   'rating': '5',
   'votes': '6'}],
 'salesrank': '396585',
 'similar': ['0804215715',
  '156101074X',
  '0687023955',
  '0687074231',
  '082721619X'],
 'title': 'Patterns of Preaching: A Sermon Sampler'}

#1) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação


In [ ]:
p = products.find_one({'id': '2'})

v = sorted(p['reviews'], key=lambda x: (int(x['helpful']), int(x['rating'])),reverse=True)[0:5]
v = sorted(v, key=lambda x: (x['rating']),reverse=True)

print("5 comentários mais úteis ordenados pela maior avaliação")
for i in v:
  print(i)

print("\n5 comentários mais úteis ordenados pela menor avaliação")
v = sorted(v, key=lambda x: (x['rating']))
for i in v:
  print(i)

5 comentários mais úteis ordenados pela maior avaliação
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-5-23', 'cliente': 'A1GIL64QK68WKL', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'}
{'data': '2004-2-11', 'cliente': 'A1CP26N8RHYVVO', 'rating': '1', 'votes': '13', 'helpful': '9'}

5 comentários mais úteis ordenados pela menor avaliação
{'data': '2004-2-11', 'cliente': 'A1CP26N8RHYVVO', 'rating': '1', 'votes': '13', 'helpful': '9'}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'}
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'}
{'data': '2002-5-23

#2) Dado um produto, listar os produtos similares com maiores vendas do que ele

In [ ]:
p = products.find_one({'id': '2'})

c = products.find({
        '$and': [
            {'asin': {'$in': p['similar']}},
            {'salesrank': {'$lt': p['salesrank']}}
        ] 
    }, {
        'id': 1,
        'title': 1,
        'salesrank': 1,
        '_id': 0
    })

for i in list(c):
  print(i)

{'id': '62291', 'title': 'Yule: A Celebration of Light and Warmth', 'salesrank': '103012'}
{'id': '170507', 'title': 'Midsummer: Magical Celebrations of the Summer Solstice', 'salesrank': '159277'}


#3) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

In [ ]:
def media(text): #media de diaas
  vet = {'media':[]}
  rate = []
  cont = 1
  for review in text['reviews']:
    rate.append(float(review['rating']))
    if cont == 1:
      vet['media'].append({'media_evolucao': float(review['rating'])})       
    else:
      vet['media'].append({'media_evolucao': (sum(rate)) / cont}) 

    cont+=1
  return vet['media']

p = products.find_one({'id': '2'}, {'_id': 0, 'id': 1, 'reviews': 1})
x = media(p)
cont = 0
for key, value in p.items():
  for i in value:
    if i != '2':
      print(i, x[cont])
      cont+=1


{'data': '2001-12-16', 'cliente': 'A11NCO6YTE4BTJ', 'rating': '5', 'votes': '5', 'helpful': '4'} {'media_evolucao': 5.0}
{'data': '2002-1-7', 'cliente': 'A9CQ3PLRNIR83', 'rating': '4', 'votes': '5', 'helpful': '5'} {'media_evolucao': 4.5}
{'data': '2002-1-24', 'cliente': 'A13SG9ACZ9O5IM', 'rating': '5', 'votes': '8', 'helpful': '8'} {'media_evolucao': 4.666666666666667}
{'data': '2002-1-28', 'cliente': 'A1BDAI6VEYMAZA', 'rating': '5', 'votes': '4', 'helpful': '4'} {'media_evolucao': 4.75}
{'data': '2002-2-6', 'cliente': 'A2P6KAWXJ16234', 'rating': '4', 'votes': '16', 'helpful': '16'} {'media_evolucao': 4.6}
{'data': '2002-2-14', 'cliente': 'AMACWC3M7PQFR', 'rating': '4', 'votes': '5', 'helpful': '5'} {'media_evolucao': 4.5}
{'data': '2002-3-23', 'cliente': 'A3GO7UV9XX14D8', 'rating': '4', 'votes': '6', 'helpful': '6'} {'media_evolucao': 4.428571428571429}
{'data': '2002-5-23', 'cliente': 'A1GIL64QK68WKL', 'rating': '5', 'votes': '8', 'helpful': '8'} {'media_evolucao': 4.5}
{'data': '20

#4) Listar os 10 produtos líderes de venda em categoria

In [ ]:
p = products.find({'categories':{'$in': ["Criminology[11005]"]}}, {'_id': 0, 'id': 1, 'title':1, 'salesrank':1}).sort('salesrank',-1)[0:10]

for i in list(p):
  print(i)

{'id': '517628', 'title': 'Prosecuting and Defending Health Care Fraud Cases', 'salesrank': '997450'}
{'id': '217253', 'title': 'Mapping and Analysing Crime Data', 'salesrank': '996848'}
{'id': '9374', 'title': 'With Liberty for Some: 500 Years of Imprisonment in America', 'salesrank': '995575'}
{'id': '507767', 'title': 'Taking Back Our Lives : A Call to Action for the Feminist Movement', 'salesrank': '995027'}
{'id': '295', 'title': 'Inventing Criminology: Essays on the Rise of Homo Criminalis (S U N Y Series in Deviance and Social Control)', 'salesrank': '994146'}
{'id': '428346', 'title': 'Criminology at the Crossroads: Feminist Readings in Crime and Justice (Readings in Criminology and Criminal Justice)', 'salesrank': '991014'}
{'id': '192060', 'title': 'Studying and Preventing Homicide : Issues and Challenges', 'salesrank': '990132'}
{'id': '44812', 'title': "The Pimp's Bible: The Sweet Science of Sin", 'salesrank': '9890'}
{'id': '54392', 'title': 'Violence and Psychopathy', 'sa

#5) Listar os 10 produtos com a maior média de ratings por grupo (Group) de produtos

In [ ]:
groups = products.distinct('group')

for i in groups:
  p = products.find({'group':i}, {'_id': 0, 'title': 1, 'group':1, 'avg_rating':1}).sort('avg_rating',-1)[0:10]
  print(i)
  for j in list(p):
    print(j)

Book
{'title': 'Patterns of Preaching: A Sermon Sampler', 'group': 'Book', 'avg_rating': '5'}
{'title': 'World War II Allied Fighter Planes Trading Cards', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Resetting the Clock : Five Anti-Aging Hormones That Improve and Extend Life', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Computed Tomography : Fundamentals, System Technology, Image Quality, Applications', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Backroad Buffets & Country Cafes: A Southern Guide to Meat-And-Threes & Down-Home Dining', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Double Jeopardy (T*Witches, 6)', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Renovations: A Father and Son Rebuild a House and Rediscover Each Other', 'group': 'Book', 'avg_rating': '5'}
{'title': 'How to Kill a Monster (Goosebumps)', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Baby Boomer Retirement: 65 Simple Ways to Protect Your Future', 'group': 'Book', 'avg_rating': '5'}
{'title': 'Betsy 

#6) Listar a 5 categorias de produto com a maior média geral de ratings 

In [ ]:
p = products.find({'avg_rating':'5'}, {'_id': 0,  'categories':1, 'avg_rating':1}).sort('avg_rating',-1)[0:5]

p = list(p)

for i in p:
  print(i)


{'categories': ['Preaching[12368]', 'Sermons[12370]'], 'avg_rating': '5'}
{'categories': ['General[5144]'], 'avg_rating': '5'}
{'categories': ['General[4701]', 'General[14046]', 'General[227584]', 'jp-unknown2[1061350]'], 'avg_rating': '5'}
{'categories': ['General[63926]'], 'avg_rating': '5'}
{'categories': ['General[14407]', 'General[227709]', 'General[657762]'], 'avg_rating': '5'}


#7) Listar os 10 clientes que mais fizeram comentários por grupo (Group) de produto

In [ ]:
groups = products.distinct('group')

for i in groups:
  p = products.find({'group':i}, {'_id': 0, 'group': 1, 'reviews':1})
  
  vet = {}

  for product in p:
    for r in product['reviews']:
      if r['cliente'] in vet:
        vet[r['cliente']] = vet[r['cliente']] + 1
      else:
        vet[r['cliente']] = 1
  print(i)
  x = sorted(vet.items(), key=lambda x: x[1], reverse=True)[0:10]
  for j in x:
    print(j)


Book
('ATVPDKIKX0DER', 643185)
('A3UN6WX5RRO2AG', 154531)
('A14OJS0VWMOSWO', 9589)
('AFVQZQ8PW0L', 5441)
('A1K1JW1C5CUSUZ', 3562)
('A2NJO6YE954DBH', 2055)
('A3QVAKVRAH657N', 1651)
('A1NATT3PN24QWY', 1535)
('A1D2C0WDCSHUWZ', 1508)
('A2ODBHT4URXVXQ', 1469)
Music
('ATVPDKIKX0DER', 166149)
('A3UN6WX5RRO2AG', 15875)
('A9Q28YTLYREO7', 2760)
('A2U49LUUY4IKQQ', 1258)
('A1GN8UJIZLCA59', 1154)
('A2NJO6YE954DBH', 1128)
('A1J5KCZC8CMW9I', 1031)
('A3MOF5KF93Q6WE', 989)
('AXFI7TAWD6H6X', 814)
('A38U2M9OAEJAXJ', 780)
DVD
('ATVPDKIKX0DER', 63148)
('A3UN6WX5RRO2AG', 15549)
('A2NJO6YE954DBH', 1366)
('AU8552YCOO5QX', 1213)
('A3P1A63Q8L32C5', 859)
('A3LZGLA88K0LA0', 856)
('A82LIVYSX6WZ9', 683)
('A152C8GYY25HAH', 675)
('A16CZRQL23NOIW', 651)
('A1CZICCYP2M5PX', 650)
Video
('ATVPDKIKX0DER', 72581)
('A3UN6WX5RRO2AG', 15814)
('A2NJO6YE954DBH', 1775)
('AU8552YCOO5QX', 1205)
('A3P1A63Q8L32C5', 737)
('A20EEWWSFMZ1PN', 720)
('A16CZRQL23NOIW', 668)
('A3LZGLA88K0LA0', 614)
('A2QRB6L1MCJ53G', 606)
('A152C8GYY25HAH', 